# Compare a text-based vs a vision-based browser

Warning: this notebook is experimental, it probably won't work out of the box!

In [23]:
!pip install "smolagents[litellm]" -q


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd

eval_df = pd.read_json("GAIA_web.jsonl", lines=True)

In [4]:
import os

from dotenv import load_dotenv
from huggingface_hub import login
from scripts.reformulator import prepare_response


load_dotenv(override=True)

login(os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### Text browser

In [12]:
from scripts.text_inspector_tool import TextInspectorTool
from scripts.text_web_browser import (
    ArchiveSearchTool,
    FinderTool,
    FindNextTool,
    PageDownTool,
    PageUpTool,
    SearchInformationTool,
    VisitTool,
    SimpleTextBrowser,
)
from scripts.visual_qa import visualizer

from smolagents import CodeAgent, LiteLLMModel, GoogleSearchTool


model = LiteLLMModel("gpt-4o")

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0"
BROWSER_CONFIG = {
    "viewport_size": 1024 * 5,
    "downloads_folder": "downloads_folder",
    "request_kwargs": {
        "headers": {"User-Agent": user_agent},
        "timeout": 300,
    },
    "serpapi_key": os.getenv("SERPAPI_API_KEY"),
}

os.makedirs(f"./{BROWSER_CONFIG['downloads_folder']}", exist_ok=True)

browser = SimpleTextBrowser(**BROWSER_CONFIG)

WEB_TOOLS = [
    GoogleSearchTool(provider="serper"),
    VisitTool(browser),
    PageUpTool(browser),
    PageDownTool(browser),
    FinderTool(browser),
    FindNextTool(browser),
    ArchiveSearchTool(browser),
]


surfer_agent = CodeAgent(
    model=model,
    tools=WEB_TOOLS,
    max_steps=20,
    verbosity_level=1,
)

answers_text = []

for example in eval_df.to_dict(orient="index").values():
    try:
        answer = surfer_agent.run(example["ques"])
        final_answer = prepare_response(
            example["ques"], f"Here is the final report from the agent: {answer}", reformulation_model=model
        )

        example["prediction"] = final_answer
        answers_text.append(example)
    except Exception as e:
        print(e)

import json

with open("gaia_text.jsonl", "w") as f:
    for item in answers_text:
        json_line = json.dumps(item)
        f.write(json_line + "\n")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ If Eliud Kipchoge could maintain his record-making marathon pace indefinitely, how many thousand hours would it │
│ take him to run the distance between the Earth and the Moon its closest approach? Please use the minimum        │
│ perigee value on the Wikipedia page for the Moon when carrying out your calculation. Round your result to the   │
│ nearest 1000 hours and do not use any comma separators if necessary.                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Eliud Kipchoge's marathon pace details                                                                         
  marathon_distance_km = 42.195  # Marathon distance in kilometers                                                 
  kipchoge_marathon_time_hours = 2 + 1/60  # 2 hours 1 minute in hours                                             
  kipchoge_speed_kmph = marathon_distance_km / kipchoge_marathon_time_hours                                        
                                                                                                                   
  # Print Kipchoge's average speed                                                                                 
  kipchoge_speed_kmph                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out: 20.92314049586777

[Step 1: Duration 3.88 seconds| Input tokens: 2,509 | Output tokens: 229]

...

[Step 4: Duration 1.93 seconds| Input tokens: 14,255 | Output tokens: 309]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import re                                                                                                        
                                                                                                                   
  # Initializing counters for pagination and 'ps' version availability                                             
  ps_available_count = 0                                                                                           
  page_counter = 0                                                                                                 
                                                                                                                   
  while True:                                                                                                      
      # Visit the page for each block of 50 articles                                                               
      url_block = f"https://arxiv.org/list/hep-lat/2020-01?skip={page_counter}&show=50"                            
      page_content = visit_page(url=url_block)                                                                     
                                                                                                                   
      # Find entries with 'other' formats available                                                                
      matches = re.findall(r'\[other\]\(/format/(\d+\.\d+)\s?\"Other formats\"\)', page_content)                   
                                                                                                                   
      # Check each entry's formats                                                                                 
      for entry_code in matches:                                                                                   
          format_page = visit_page(url=f"https://arxiv.org/format/{entry_code}")                                   
          if 'PostScript' in format_page or 'ps' in format_page:                                                   
              ps_available_count += 1                                                                              
                                                                                                                   
      # Check if there's a next page by counting the number of entries processed                                   
      if 'skip' not in page_content:                                                                               
          break                                                                                                    
                                                                                                                   
      # Move to the next page of 50 articles                                                                       
      page_counter += 50                                                                                           
                                                                                                                   
  print(f"Total number of articles with 'ps' versions available: {ps_available_count}")                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

[Step 5: Duration 7298.75 seconds| Input tokens: 21,613 | Output tokens: 1,065]

KeyboardInterrupt: 

In [13]:
import json

with open("gaia_text.jsonl", "w") as f:
    for item in answers_text:
        json_line = json.dumps(item)
        f.write(json_line + "\n")

### Vision browser

In [ ]:
!pip install helium -q

In [ ]:
from scripts.visual_qa import VisualQAGPT4Tool

from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel
from smolagents.vision_web_browser import (
    close_popups,
    go_back,
    helium_instructions,
    initialize_agent,
    save_screenshot,
    search_item_ctrl_f,
)


proprietary_model = LiteLLMModel("gpt-4o")
vision_browser_agent = initialize_agent(proprietary_model)
### BUILD AGENTS & TOOLS

CodeAgent(
    tools=[DuckDuckGoSearchTool(), go_back, close_popups, search_item_ctrl_f],
    model=proprietary_model,
    additional_authorized_imports=["helium"],
    step_callbacks=[save_screenshot],
    max_steps=20,
    verbosity_level=2,
)

results_vision = answer_questions(
    eval_ds,
    vision_browser_agent,
    "code_gpt4o_27-01_vision",
    reformulation_model=proprietary_model,
    output_folder="output_browsers",
    visual_inspection_tool=VisualQAGPT4Tool(),
    text_inspector_tool=TextInspectorTool(proprietary_model, 40000),
    postprompt=helium_instructions
    + "Any web browser controls won't work on .pdf urls, rather use the tool 'inspect_file_as_text' to read them",
)

### Browser-use browser

In [ ]:
!pip install browser-use lxml_html_clean -q
!playwright install -q

In [ ]:
import asyncio

import nest_asyncio


nest_asyncio.apply()

from browser_use import ActionResult, Agent, Browser, Controller
from browser_use.browser.context import BrowserContext, BrowserContextConfig
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from smolagents import GoogleSearchTool, LiteLLMModel


load_dotenv()


config = BrowserContextConfig(
    wait_for_network_idle_page_load_time=3.0,
    browser_window_size={"width": 1280, "height": 1000},
    locale="en-US",
    user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36",
    highlight_elements=True,
    viewport_expansion=500,
)

browser = Browser()
context = BrowserContext(browser=browser, config=config)

model = LiteLLMModel("gpt-4o")

# Initialize the controller
controller = Controller()


@controller.action("Use this to run google searches")
def search_google(question: str) -> str:
    answer = GoogleSearchTool(provider="serper")(question)
    return ActionResult(extracted_content=answer)


class BrowserUseAgent:
    logs = []

    def write_inner_memory_from_logs(self, summary_mode):
        return self.results

    def run(self, task, **kwargs):
        agent = Agent(
            browser_context=context,
            task=task,
            llm=ChatOpenAI(model="gpt-4o"),
            controller=controller,
        )
        self.results = asyncio.get_event_loop().run_until_complete(agent.run())
        return self.results.history[-1].result[0].extracted_content


browser_use_agent = BrowserUseAgent()

answers_browser_use = []

for example in eval_df.to_dict(orient="index").values():
    try:
        answer = browser_use_agent.run(
            example["ques"] + "Use your search_google action rather than browsing google", max_steps=30
        )
        final_answer = prepare_response(
            example["ques"], f"Here is the final report from the agent: {answer}", reformulation_model=model
        )

        example["prediction"] = final_answer
        answers_browser_use.append(example)
    except Exception as e:
        print(e)

import json

with open("gaia_browser-use.jsonl", "w") as f:
    for item in answers_browser_use:
        json_line = json.dumps(item)
        f.write(json_line + "\n")

INFO     [agent] 🚀 Starting task: In terms of geographical distance between capital cities, which 2 countries are the furthest from each other within the ASEAN bloc according to wikipedia? Answer using a comma separated list, ordering the countries by alphabetical order.Use your search_google action rather than browsing google
INFO     [agent] 
📍 Step 1
INFO     [agent] 🤷 Eval: Unknown - Starting with a blank page, need to initiate a search.
INFO     [agent] 🧠 Memory: 
INFO     [agent] 🎯 Next goal: Search for information on the geographical distance between ASEAN capitals.
INFO     [agent] 🛠️  Action 1/1: {"search_google":{"question":"which 2 countries are the furthest from each other within the ASEAN bloc according to wikipedia"}}
INFO     [agent] 
📍 Step 2
INFO     [agent] 👍 Eval: Success - Retrieved search results with relevant links to ASEAN and Wikipedia.
INFO     [agent] 🧠 Memory: Need to find the information on geographical distance between ASEAN capitals.
INFO     [agent] 🎯 Nex

14:51:02 - LiteLLM:INFO: utils.py:2944 - 
LiteLLM completion() model= gpt-4o; provider = openai


INFO     [LiteLLM] 
LiteLLM completion() model= gpt-4o; provider = openai


14:51:03 - LiteLLM:INFO: utils.py:1120 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler
> Reformulated answer:  Myanmar, Philippines
INFO     [agent] 🚀 Starting task: I need to fact-check a citation. This is the citation from the bibliography:

Greetham, David. "Uncoupled: OR, How I Lost My Author(s)." Textual Cultures: Texts, Contexts, Interpretation, vol. 3 no. 1, 2008, p. 45-46. Project MUSE, doi:10.2979/tex.2008.3.1.44.

And this is the in-line citation:

Our relationship with the authors of the works we read can often be “obscured not by a "cloak of print" but by the veil of scribal confusion and mis-transmission” (Greetham 45-46).

Does the quoted text match what is actually in the article? If Yes, answer Yes, otherwise, give me the word in my citation that does not match with the correct one (without any article).Use your search_google action rather than browsing google
INFO     [agent] 
📍 Step 1
INFO     [agent] 👍 Eval: Success - Browser started successfully.
INFO     [agent] 🧠 Memory: Started bro

...

14:55:54 - LiteLLM:INFO: utils.py:2944 - 
LiteLLM completion() model= gpt-4o; provider = openai


INFO     [LiteLLM] 
LiteLLM completion() model= gpt-4o; provider = openai


14:55:55 - LiteLLM:INFO: utils.py:1120 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler
> Reformulated answer:  Unable to determine
INFO     [agent] 🚀 Starting task: What is the maximum length in meters of #9 in the first National Geographic short on YouTube that was ever released according to the Monterey Bay Aquarium website? Just give the number.Use your search_google action rather than browsing google
INFO     [agent] 
📍 Step 1
ERROR    [agent] ❌ Result failed 1/3 times:
 Browser closed: no valid pages available
INFO     [agent] 
📍 Step 1
ERROR    [agent] ❌ Result failed 2/3 times:
 Browser closed: no valid pages available
INFO     [agent] 
📍 Step 1
ERROR    [agent] ❌ Result failed 3/3 times:
 Browser closed: no valid pages available
ERROR    [agent] ❌ Stopping due to 3 consecutive failures
WARNING  [agent] No history to create GIF from


IndexError: list index out of range

In [ ]:
import json

with open("gaia_browser-use2.jsonl", "w") as f:
    for item in list(previous_answers.to_dict(orient="index").values()) + answers_browser_use:
        json_line = json.dumps(item)
        f.write(json_line + "\n")

### Omniparser vision agent

In [2]:
!pip install ultralytics easyocr paddleocr paddlepaddle numpy==1.26.4 supervision==0.18.0 -q


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from scripts.vision_web_browser import make_vision_agent
from scripts.visual_qa import visualizer
import pandas as pd
from smolagents import LiteLLMModel

instructions = """
Use your web_search tool when you want to get Google search results.
Then you can use helium to access websites. Don't use helium for Google search, only for navigating websites!
Don't bother about the helium driver, it's already managed.
We've already ran "from helium import *"

You can navigate to pages.
Code:
```py
go_to('github.com/trending')
```<end_code>

You can directly click a box by giving its number
Code:
```py
click_box(18)
```<end_code>

In general stop your action after each button click to see what happens on your screenshot.
Never try to login in a page.

To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.
Code:
```py
scroll_down(num_pixels=1200) # This will scroll one viewport down
```<end_code>

When you have pop-ups with a cross icon to close, don't try to click the close icon by finding its element or targeting an 'X' element (this most often fails).
Just use your built-in tool `close_popups` to close them:
Code:
```py
close_popups()
```<end_code>

You can use .exists() to check for the existence of an element. For example:
Code:
```py
if Text('Accept cookies?').exists():
    click('I accept')
```<end_code>

Proceed in several steps rather than trying to solve the task in one shot.
And at the end, only when you have your answer, return your final answer.
Code:
```py
final_answer("YOUR_ANSWER_HERE")
```<end_code>

If pages seem stuck on loading, you might have to wait, for instance `import time` and run `time.sleep(5.0)`. But don't overuse this!
To list elements on page, DO NOT try code-based element searches like 'contributors = find_all(S("ol > li"))': just look at the latest screenshot you have and read it visually, or use your tool search_item_ctrl_f.
Of course, you can act on buttons like a user would do when navigating.
After each code blob you write, you will be automatically provided with an updated screenshot of the browser and the current browser url.
But beware that the screenshot will only be taken at the end of the whole action, it won't see intermediate states.
Don't kill the browser.
When you have modals or cookie banners on screen, you should get rid of them before you can click anything else.
Any web browser controls won't work on .pdf urls, rather use the tool 'inspect_file_as_text' to read them
"""

vision_browser_agent = make_vision_agent()

model = LiteLLMModel("gpt-4o")

results = []
for example in eval_ds:
    example["question"] += instructions
    answer = answer_single_question(example, model, "out.jsonl", visualizer)
    results += answer

results_omniparser = pd.DataFrame(results)

### Get results

In [16]:
import pandas as pd
from scripts.gaia_scorer import question_scorer

results_browseruse = pd.read_json("gaia_browser-use.jsonl", lines=True)
results_browseruse["agent_name"] = "browser-use"
results_browseruse["is_correct"] = results_browseruse.apply(
    lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1
)

results_text = pd.read_json("gaia_text.jsonl", lines=True)
results_text["agent_name"] = "text"
results_text["is_correct"] = results_text.apply(lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1)

String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.


/Users/aymeric/Documents/Code/smolagents/examples/open_deep_research/scripts/gaia_scorer.py:52: UserWarning: Answer lists have different lengths, returning False.
  warnings.warn("Answer lists have different lengths, returning False.", UserWarning)
/Users/aymeric/Documents/Code/smolagents/examples/open_deep_research/scripts/gaia_scorer.py:52: UserWarning: Answer lists have different lengths, returning False.
  warnings.warn("Answer lists have different lengths, returning False.", UserWarning)


In [18]:
results = pd.concat([results_text, results_browseruse])

display(results.groupby("agent_name")["is_correct"].count())
display(results.groupby("agent_name")["is_correct"].mean())

agent_name
browser-use    36
text           41
Name: is_correct, dtype: int64

agent_name
browser-use    0.333333
text           0.414634
Name: is_correct, dtype: float64

In [ ]:
correct_vision_results = results_vision.loc[results_vision["is_correct"]]
correct_vision_results

In [ ]:
false_text_results = results_text.loc[~results_text["is_correct"]]
false_text_results